In [12]:
import pandas as pd

normal = pd.read_csv('/home/work/Climate_map/CATCH/IMU_data/Beanpicking/Beanpicking_imu_data_250Hz.csv', skiprows=1, on_bad_lines="skip")  
abnormal0 = pd.read_csv('/home/work/Climate_map/CATCH/IMU_data/Beanpicking/Beanpicking_imu_data_250Hz(500g).csv',skiprows=1, on_bad_lines="skip")
abnormal1 = pd.read_csv('/home/work/Climate_map/CATCH/IMU_data/Beanpicking/Beanpicking_imu_data_250Hz(1kg).csv', skiprows=1, on_bad_lines="skip")
abnormal2 = pd.read_csv('/home/work/Climate_map/CATCH/IMU_data/Beanpicking/Beanpicking_imu_data_250Hz(2kg).csv', skiprows=1, on_bad_lines="skip")
abnormal3 = pd.read_csv('/home/work/Climate_map/CATCH/IMU_data/Beanpicking/Beanpicking_imu_data_250Hz(inner).csv', skiprows=1, on_bad_lines="skip")
abnormal4 = pd.read_csv('/home/work/Climate_map/CATCH/IMU_data/Beanpicking/Beanpicking_imu_data_250Hz(outer).csv', skiprows=1, on_bad_lines="skip")


In [13]:
# import numpy as np
# import pandas as pd

# def IMU_to_CATCH(a_0: pd.DataFrame,
#                                a_1: pd.DataFrame,
#                                out_path: str,
#                                time_col: str = "timestamp",
#                                cumulative_date: bool = False) -> pd.DataFrame:
#     """
#     a_0, a_1의 열 셋이 같다는 전제하에,
#     '컬럼 단위'로 묶어 (col별로 a_0 전부 → a_1 전부) 쌓아
#     long 포맷(date, data, cols)으로 저장.

#     - 행 수는 달라도 OK
#     - date: 기본은 각 블록(a_0, a_1)에서 1..N 재시작
#             cumulative_date=True면 a_1이 a_0의 다음 번호로 이어짐
#     - 마지막에 label 컬럼(문자 그대로)을 추가:
#         a_0 길이만큼 0, a_1 길이만큼 1
#     """
#     def value_cols(df: pd.DataFrame) -> list:
#         cols = df.columns.tolist()
#         if time_col in cols:
#             cols.remove(time_col)
#         return cols

#     cols0, cols1 = value_cols(a_0), value_cols(a_1)
#     if set(cols0) != set(cols1):
#         raise ValueError(
#             "Columns must match.\n"
#             f"Missing in a_1: {[c for c in cols0 if c not in cols1]}\n"
#             f"Missing in a_0: {[c for c in cols1 if c not in cols0]}"
#         )

#     # a_0의 컬럼 순서를 기준으로
#     cols = cols0
#     a0_len, a1_len = len(a_0), len(a_1)

#     parts = []

#     for col in cols:
#         s0 = a_0[col].to_numpy()
#         s1 = a_1[col].to_numpy()

#         if cumulative_date:
#             date0 = np.arange(1, a0_len + 1)
#             date1 = np.arange(a_0.shape[0] + 1, a_0.shape[0] + a_1.shape[0] + 1)
#         else:
#             date0 = np.arange(1, a0_len + 1)
#             date1 = np.arange(1, a1_len + 1)

#         df0 = pd.DataFrame({"date": date0, "data": s0, "cols": col})
#         df1 = pd.DataFrame({"date": date1, "data": s1, "cols": col})
#         parts.extend([df0, df1])  # col 하나에 대해 a_0 전체 → a_1 전체

#     # label 컬럼 추가 (문자 그대로 'label')
#     if cumulative_date:
#         l0_date = np.arange(1, a0_len + 1)
#         l1_date = np.arange(a0_len + 1, a0_len + a1_len + 1)
#     else:
#         l0_date = np.arange(1, a0_len + 1)
#         l1_date = np.arange(1, a1_len + 1)

#     label0 = pd.DataFrame({"date": l0_date, "data": 0, "cols": "label"})
#     label1 = pd.DataFrame({"date": l1_date, "data": 1, "cols": "label"})

#     final_df = pd.concat(parts + [label0, label1], ignore_index=True)
#     final_df.to_csv(out_path, index=False, encoding="utf-8-sig")
#     return final_df


In [2]:
import numpy as np
import pandas as pd

def IMU_to_CATCH(a_0: pd.DataFrame,
                 a_1: pd.DataFrame,
                 out_path: str,
                 time_col: str = "timestamp") -> pd.DataFrame:
    """
    두 DF(a_0, a_1)의 컬럼명이 동일하다는 전제하에,
    (1) 컬럼명 사전순으로 정렬,
    (2) 각 컬럼별로 a_0 전구간을 먼저, 이어서 a_1 전구간을 쌓고,
    (3) date 인덱스는 a_0→a_1로 '연속'(1..len(a_0)+len(a_1))이 되도록 부여,
    (4) 마지막에 label(문자 그대로) 행 추가: a_0 구간 0, a_1 구간 1,
    형태의 long 포맷(date, data, cols) CSV를 저장한다.
    """
    # 값 컬럼 선택(시간열이 있으면 제외)
    def value_cols(df: pd.DataFrame) -> list:
        cols = [c for c in df.columns if c != time_col]
        # 컬럼명 사전순 정렬
        cols.sort()
        return cols

    cols0 = value_cols(a_0)
    cols1 = value_cols(a_1)
    if set(cols0) != set(cols1):
        raise ValueError(
            "Columns must match.\n"
            f"Missing in a_1: {[c for c in cols0 if c not in cols1]}\n"
            f"Missing in a_0: {[c for c in cols1 if c not in cols0]}"
        )

    cols = cols0  # 정렬된 공통 컬럼 목록
    n0, n1 = len(a_0), len(a_1)
    total_len = n0 + n1

    # 연속 date 인덱스 (전 컬럼 공통)
    date0 = np.arange(1, n0 + 1)                 # a_0 구간
    date1 = np.arange(n0 + 1, n0 + n1 + 1)       # a_1 구간

    parts = []
    for col in cols:
        s0 = a_0[col].to_numpy()
        s1 = a_1[col].to_numpy()
        df0 = pd.DataFrame({"date": date0, "data": s0, "cols": col})
        df1 = pd.DataFrame({"date": date1, "data": s1, "cols": col})
        parts.extend([df0, df1])  # (a_0 → a_1) 순서

    # label 컬럼 추가 (문자 그대로 'label')
    label0 = pd.DataFrame({"date": date0, "data": 0, "cols": "label"})
    label1 = pd.DataFrame({"date": date1, "data": 1, "cols": "label"})

    final_df = pd.concat(parts + [label0, label1], ignore_index=True)
    final_df.to_csv(out_path, index=False, encoding="utf-8-sig")
    return final_df


In [20]:
# a_0, a_1이 이미 로드된 상태라고 가정
df_long =  IMU_to_CATCH(normal, abnormal2, out_path="/home/work/Climate_map/CATCH/dataset/anomaly_detect/data/IMU.csv", time_col="timestamp")
print(df_long.head())


   date      data     cols
0     1 -0.000000  accel_x
1     2  0.130892  accel_x
2     3  0.130892  accel_x
3     4  0.138058  accel_x
4     5  0.107008  accel_x


In [16]:
df_long

,date,data,cols
0,1,-0.000000,accel_x
1,2,0.130892,accel_x
2,3,0.130892,accel_x
3,4,0.138058,accel_x
4,5,0.107008,accel_x
...,...,...,...
9744028,749537,1.000000,label
9744029,749538,1.000000,label
9744030,749539,1.000000,label
9744031,749540,1.000000,label


In [33]:
df_long

,date,data,cols
0,1,-0.000000,accel_x
1,2,0.061626,accel_x
2,3,0.080734,accel_x
3,4,0.161943,accel_x
4,5,0.145223,accel_x
...,...,...,...
2338591,71973,1.000000,label
2338592,71974,1.000000,label
2338593,71975,1.000000,label
2338594,71976,1.000000,label


In [21]:
import tarfile
import pandas as pd

# 파일 경로
tar_path = "/home/work/Climate_map/CATCH/result/label/CATCH/CATCH.1755567637.main1.3739625.csv.tar.gz"

# tar.gz 열기
with tarfile.open(tar_path, "r:gz") as tar:
    # 압축 안에 있는 파일 목록 확인
    members = tar.getnames()
    print("압축 파일 안:", members)

    # 첫 번째 CSV 파일만 읽어오기 (여러 개면 원하는 파일 선택)
    csv_name = members[0]
    with tar.extractfile(csv_name) as f:
        df = pd.read_csv(f)

        

print(df.head())


압축 파일 안: ['CATCH.1755567637.main1.3739625.csv']
  model_name                                      strategy_args  \
0      CATCH  {"seed": 2021, "strategy_name": "unfixed_detec...   

                                        model_params  accuracy   f_score  \
0  {"Mlr": 1e-05, "anomaly_ratio": 5.0, "auxi_lam...  0.456407  0.090786   

   precision    recall  adjust_accuracy  adjust_f_score  adjust_precision  \
0   0.519706  0.049737         0.974919        0.977534          0.956055   

   ...  affiliation_f  affiliation_precision  affiliation_recall  file_name  \
0  ...       0.772598               0.629544            0.999781    IMU.csv   

     fit_time  inference_time  typical_anomaly_ratio  actual_data  \
0  974.271445      926.126641                    5.0          NaN   

  inference_data  log_info  
0            NaN       NaN  

[1 rows x 25 columns]


In [24]:
df

,model_name,strategy_args,model_params,accuracy,f_score,precision,recall,adjust_accuracy,adjust_f_score,adjust_precision,...,affiliation_f,affiliation_precision,affiliation_recall,file_name,fit_time,inference_time,typical_anomaly_ratio,actual_data,inference_data,log_info
0,CATCH,"{""seed"": 2021, ""strategy_name"": ""unfixed_detec...","{""Mlr"": 1e-05, ""anomaly_ratio"": 5.0, ""auxi_lam...",0.456407,0.090786,0.519706,0.049737,0.974919,0.977534,0.956055,...,0.772598,0.629544,0.999781,IMU.csv,974.271445,926.126641,5.0,NaN,NaN,NaN


In [ ]:
sh ./scripts/multivariate_detection/detect_label/IMU_script/CATCH.sh